In [164]:
import pandas as pd
from config import *
import os
import numpy as np

In [165]:
def get_market_year(row): #make a market year row in datas
    year = row['YEAR']
    month = row['MONTH']
    if month >= 9:
        market_year = f"{year}/{year + 1}"
    else:
        market_year = f"{year - 1}/{year}"
    return market_year

In [166]:
departements = { #set a dict of departement name as keys and dep code as value for missing dep values in datasets
    'Ain': 1,
    'Aisne': 2,
    'Allier': 3,
    'Alpes-de-Haute-Provence': 4,
    'Hautes-Alpes': 5,
    'Alpes-Maritimes': 6,
    'Ardèche': 7,
    'Ardennes': 8,
    'Ariège': 9,
    'Aube': 10,
    'Aude': 11,
    'Aveyron': 12,
    'Bouches-du-Rhône': 13,
    'Calvados': 14,
    'Cantal': 15,
    'Charente': 16,
    'Charente-Maritime': 17,
    'Cher': 18,
    'Corrèze': 19,
    'Corse': 20,
    "Côte-d'Or": 21,
    "Côtes-d'Armor": 22,
    'Creuse': 23,
    'Dordogne': 24,
    'Doubs': 25,
    'Drôme': 26,
    'Eure': 27,
    'Eure-et-Loir': 28,
    'Finistère': 29,
    'Gard': 30,
    'Haute-Garonne': 31,
    'Gers': 32,
    'Gironde': 33,
    'Hérault': 34,
    'Ille-et-Vilaine': 35,
    'Indre': 36,
    'Indre-et-Loire': 37,
    'Isère': 38,
    'Jura': 39,
    'Landes': 40,
    'Loir-et-Cher': 41,
    'Loire': 42,
    'Haute-Loire': 43,
    'Loire-Atlantique': 44,
    'Loiret': 45,
    'Lot': 46,
    'Lot-et-Garonne': 47,
    'Lozère': 48,
    'Maine-et-Loire': 49,
    'Manche': 50,
    'Marne': 51,
    'Haute-Marne': 52,
    'Mayenne': 53,
    'Meurthe-et-Moselle': 54,
    'Meuse': 55,
    'Morbihan': 56,
    'Moselle': 57,
    'Nièvre': 58,
    'Nord': 59,
    'Oise': 60,
    'Orne': 61,
    'Pas-de-Calais': 62,
    'Puy-de-Dôme': 63,
    'Pyrénées-Atlantiques': 64,
    'Hautes-Pyrénées': 65,
    'Pyrénées-Orientales': 66,
    'Bas-Rhin': 67,
    'Haut-Rhin': 68,
    'Rhône': 69,
    'Haute-Saône': 70,
    'Saône-et-Loire': 71,
    'Sarthe': 72,
    'Savoie': 73,
    'Haute-Savoie': 74,
    'Paris': 75,
    'Seine-Maritime': 76,
    'Seine-et-Marne': 77,
    'Yvelines': 78,
    'Deux-Sèvres': 79,
    'Somme': 80,
    'Tarn': 81,
    'Tarn-et-Garonne': 82,
    'Var': 83,
    'Vaucluse': 84,
    'Vendée': 85,
    'Vienne': 86,
    'Haute-Vienne': 87,
    'Vosges': 88,
    'Yonne': 89,
    'Territoire de Belfort': 90,
    'Essonne': 91,
    'Hauts-de-Seine': 92,
    'Seine-Saint-Denis': 93,
    'Val-de-Marne': 94,
    "Val-d'Oise": 95
}

Note : We will pivot data with month number in column name. Datas starts from 1 (Jan) to 12 (Dec), but it is understood and applicated that months from 9 to 12 are harvest year - 1 and months from 1 to 8 are harvest year.

-> Market Year for EU wheat is September to Aug (for 2020 harvest, we plant in september 2019 and harvest july/aug 2020)

Weather data

In [ ]:
weather_raw = pd.read_csv(f"{WEATHER_DATA_URL}/rr_tn_tx_tm-1950-2023.csv").drop(["Unnamed: 0"], axis=1) #read raw weather data
weather_raw_2024 = pd.read_csv(f"{WEATHER_DATA_URL}/rr_tn_tx_tm-current.csv").drop(["Unnamed: 0"], axis=1) #read raw weather data

weather_raw["YEAR-MONTH"] = pd.to_datetime(weather_raw["DATE"]).dt.to_period('M') #new column with YYYY-MM format
weatherMeanGroupedDepMonth = weather_raw[["YEAR-MONTH", "DEP", "RR", "TN", "TX", "TM"]].groupby(["YEAR-MONTH", "DEP"]).mean() #group by year-month and dep then mean the values
weather = weatherMeanGroupedDepMonth.reset_index() #remove multi indexing 
weather["YEAR"] = weather["YEAR-MONTH"].dt.year
weather["MONTH"] = weather["YEAR-MONTH"].dt.month
weather["MY"] = weather.apply(get_market_year, axis=1) #add year, month and market year for merging 

weather_raw_2024 = pd.read_csv(f"{WEATHER_DATA_URL}/rr_tn_tx_tm-current.csv").drop(["Unnamed: 0"], axis=1) #read raw weather data
weather_raw_2024 = weather_raw_2024[weather_raw_2024["DATE"] < "2024-09-01"].reset_index(drop=True)
weather_raw_2024["YEAR-MONTH"] = pd.to_datetime(weather_raw_2024["DATE"]).dt.to_period('M') #new column with YYYY-MM format
weatherMeanGroupedDepMonth2024 = weather_raw_2024[["YEAR-MONTH", "DEP", "RR", "TN", "TX", "TM"]].groupby(["YEAR-MONTH", "DEP"]).mean() #group by year-month and dep then mean the values
weather2024 = weatherMeanGroupedDepMonth2024.reset_index() #remove multi indexing 
weather2024["YEAR"] = weather2024["YEAR-MONTH"].dt.year
weather2024["MONTH"] = weather2024["YEAR-MONTH"].dt.month
weather2024["MY"] = weather2024.apply(get_market_year, axis=1) #add year, month and market year for merging 

weather = pd.concat([weather, weather2024])
weather = weather[(weather["MY"] >= "1979/1980") & (weather["MY"] <= "2023/2024")] #-> final weather data

KeyboardInterrupt: 

In [169]:
weather_pivot = weather.pivot_table( # Pivot table for month datas as columns and not rows
    index=['DEP', 'MY'],
    columns='MONTH',
    values=['RR', 'TN', 'TX', 'TM']
)

weather_pivot.columns = [ # rename columns with month number
    f"{col[0]}{col[1]}" if isinstance(col, tuple) and col[1] != "" 
    else col for col in weather_pivot.columns
]
weather_pivot = weather_pivot.reset_index()

Yields data

1st dataset (1900 - 2018)

In [171]:
yields_raw = pd.read_csv(f"{YIELDS_DATA_URL}/2021-001_Schauberger-et-al_Data_FILTERED/wheat_total_data_1900-2018_FILTERED.txt", sep=';')

#aggregate Corse du sud and Haute Corse into one Corse department :
corseSud = yields_raw[yields_raw['department'] == "Corse-du-sud"].fillna(0)
corseHaute = yields_raw[yields_raw['department'] == "Haute-Corse"].fillna(0)
corseArea = corseSud["area"].reset_index(drop=True) + corseHaute["area"].reset_index(drop=True)
corseProd = corseSud["production"].reset_index(drop=True) + corseHaute["production"].reset_index(drop=True)
corseYield = (corseSud["production"].reset_index(drop=True) + corseHaute["production"].reset_index(drop=True)) / (corseSud["area"].reset_index(drop=True) + corseHaute["area"].reset_index(drop=True))
corseYears = pd.Series(range(1900, 2019))
corse = pd.DataFrame({"department": "Corse", "year": corseYears, "yield": corseYield, "area": corseArea, "production": corseProd})
yields_raw = pd.concat([yields_raw, corse])
yields_raw = yields_raw[(yields_raw['department'] != "Corse-du-sud") & (yields_raw['department'] != "Haute-Corse")]

yields_raw['DEP'] = yields_raw['department'].map(departements).replace('NA', np.nan) #map dep name to dep code
yields_raw = yields_raw.drop("department", axis=1)
yields_raw["MY"] = (yields_raw['year'] - 1).astype(str) + '/' + yields_raw['year'].astype(str) 
oldYields = yields_raw[yields_raw["year"] > 1979] #-> final yields data

2nd dataset (2000 - 2024)

In [172]:
yields_raw = pd.read_csv(f"{YIELDS_DATA_URL}/SCR-GRC-hist_dep_surface_prod_cult_cer-A25.csv", encoding='utf-8')
yields_raw["ESPECES"] = yields_raw["ESPECES"].str.strip() #remove left and white spaces
yields_raw = yields_raw[yields_raw["ESPECES"] == "Blé tendre"].reset_index(drop=True) #filter soft wheat

#Corse
corseSud = yields_raw[yields_raw["DEP"] == "2A"]
corseSud = pd.concat([pd.DataFrame({'ANNEE': list(range(2000, 2016))}), corseSud], ignore_index=True).fillna(0)
corseHaute = yields_raw[yields_raw['DEP'] == "2B"].fillna(0)
corseArea = corseSud["CULT_SURF"].reset_index(drop=True) + corseHaute["CULT_SURF"].reset_index(drop=True)
corseProd = corseSud["CULT_PROD"].reset_index(drop=True) + corseHaute["CULT_PROD"].reset_index(drop=True)
corseYield = (corseSud["CULT_PROD"].reset_index(drop=True) + corseHaute["CULT_PROD"].reset_index(drop=True)) / (corseSud["CULT_SURF"].reset_index(drop=True) + corseHaute["CULT_SURF"].reset_index(drop=True))
corseYears = pd.Series(range(2000, 2026))
corse = pd.DataFrame({"DEPARTEMENT": "Corse", "DEP": 20, "ANNEE": corseYears, "CULT_REND": corseYield, "CULT_SURF": corseArea, "CULT_PROD": corseProd})
yields_raw = pd.concat([yields_raw, corse])
yields_raw = yields_raw[(yields_raw['DEP'] != "2A") & (yields_raw['DEP'] != "2B")]

yields_raw["DEP"] = yields_raw["DEP"].astype(int)
yields = yields_raw[["ANNEE", "DEP", "CULT_SURF", "CULT_REND", "CULT_PROD"]] #keep only wanted data
yields["MY"] = (yields['ANNEE'] - 1).astype(str) + '/' + yields['ANNEE'].astype(str) 
yields = yields.rename(columns={"ANNEE": "year", "CULT_SURF": "area", "CULT_REND": "yield", "CULT_PROD": "production"})
yields['yield'] = yields['yield'] / 10 #convert to kg/ha
newYields = yields[yields["year"] <= 2024] #-> final yield data

C:\Users\alexl\AppData\Local\Temp\ipykernel_16232\1724735824.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yields["MY"] = (yields['ANNEE'] - 1).astype(str) + '/' + yields['ANNEE'].astype(str)


Concat both yields datasets

In [173]:
newYields = newYields[newYields['year'] >= 2019]
yields = pd.concat([oldYields, newYields])
yields = yields.dropna().reset_index(drop=True)
yields

,year,yield,area,production,DEP,MY
0,1980,4.39805,36001.0,158334.3,1,1979/1980
1,1981,4.30319,36075.0,155237.5,1,1980/1981
2,1982,5.00000,38050.0,190250.0,1,1981/1982
3,1983,3.29500,37924.0,124959.6,1,1982/1983
4,1984,5.90000,40000.0,236000.0,1,1983/1984
...,...,...,...,...,...,...
4027,2020,0.35000,72.0,252.0,20,2019/2020
4028,2021,0.65000,99.0,643.5,20,2020/2021
4029,2022,0.65000,83.0,539.5,20,2021/2022
4030,2023,0.40000,63.0,252.0,20,2022/2023


Vapor pressure deficit data

In [174]:
vpd_raw = pd.DataFrame(columns=["dep", "date", "vpd_max", "vpd_min", "vpd_mean"])
for vpd_file in os.listdir(f"{VPD_DATA_URL}/dailyDepDatas/"): #loop throught files
    vpd_dep = pd.read_json(f"{VPD_DATA_URL}/dailyDepDatas/{vpd_file}") #read json
    vpd_raw = pd.concat([vpd_raw, vpd_dep]) #concat data
vpd_raw['DEP'] = vpd_raw['dep'].map(departements) #map dep name to dep code
vpd_raw["YEAR-MONTH"] = pd.to_datetime(vpd_raw["date"]).dt.to_period('M') #new column with YYYY-MM format
vpdMeanGroupedDepMonth = vpd_raw[["vpd_max", "vpd_min", "vpd_mean", "YEAR-MONTH", "DEP"]].groupby(["YEAR-MONTH", "DEP"]).mean() #group by year-month and dep then mean the values
vpd = vpdMeanGroupedDepMonth.reset_index() #remove multi indexing 
vpd = vpd.dropna() #removes nan values (when nan value, there is no data for the dep)
vpd["YEAR"] = vpd["YEAR-MONTH"].dt.year
vpd["MONTH"] = vpd["YEAR-MONTH"].dt.month
vpd["MY"] = vpd.apply(get_market_year, axis=1) #add year, month and market year for merging 
vpd = vpd[vpd["MY"] >= "1979/1980"] #-> final vpd data

C:\Users\alexl\AppData\Local\Temp\ipykernel_16232\3789123208.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  vpd_raw = pd.concat([vpd_raw, vpd_dep]) #concat data


In [175]:
#Historical calculation with weather data
vpd_raw = pd.read_csv(f"{VPD_DATA_URL}/vpd_historical_1950_2024.csv")
vpd_raw["YEAR-MONTH"] = pd.to_datetime(vpd_raw["YEAR-MONTH"]).dt.to_period('M') #new column with YYYY-MM format
vpdMeanGroupedDepMonth = vpd_raw[["vpd_max", "vpd_min", "vpd_mean", "YEAR-MONTH", "DEP"]].groupby(["YEAR-MONTH", "DEP"]).mean() #group by year-month and dep then mean the values
vpd = vpdMeanGroupedDepMonth.reset_index() #remove multi indexing 
vpd = vpd.dropna() #removes nan values (when nan value, there is no data for the dep)
vpd["YEAR"] = vpd["YEAR-MONTH"].dt.year
vpd["MONTH"] = vpd["YEAR-MONTH"].dt.month
vpd["MY"] = vpd.apply(get_market_year, axis=1) #add year, month and market year for merging 

In [176]:
vpd_pivot = vpd.pivot_table( # pivot table for month datas as columns and not rows
    index=['DEP', 'MY'],
    columns='MONTH',
    values=['vpd_mean', 'vpd_min', 'vpd_max']
)

vpd_pivot.columns = [ # rename columns with month number
    f"{col[0]}{col[1]}" if isinstance(col, tuple) and col[1] != "" 
    else col for col in vpd_pivot.columns
]
vpd_pivot = vpd_pivot.reset_index()

Enhanced Vegetation Index data

In [177]:
evi = pd.DataFrame(columns=["name", "code", "date", "mean_data"]) 
for evi_file in os.listdir(f"{EVI_DATA_URL}/monthlyDepMean/"): #loop throught files
    evi_dep = pd.read_json(f"{EVI_DATA_URL}/monthlyDepMean/{evi_file}") #read json
    evi = pd.concat([evi, evi_dep]) #concat data
evi["YEAR-MONTH"] = pd.to_datetime(evi["date"]).dt.to_period('M') #set period (already to monthly data but we keep same format for every dataset (Year-Month))
evi = evi.rename(columns={"code": "DEP", "mean_data": "evi"}) #rename for same format
evi = evi[["YEAR-MONTH", "DEP", "evi"]].sort_values(by="YEAR-MONTH") #keep wanted data 
evi["YEAR"] = evi["YEAR-MONTH"].dt.year
evi["MONTH"] = evi["YEAR-MONTH"].dt.month
evi["MY"] = evi.apply(get_market_year, axis=1) #add year, month and market year for merging 
evi = evi[evi["MY"] >= "2000/2001"] #-> final evi data

C:\Users\alexl\AppData\Local\Temp\ipykernel_16232\33089076.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  evi = pd.concat([evi, evi_dep]) #concat data


In [178]:
evi_pivot = evi.pivot_table( # pivot table for month datas as columns and not rows
    index=['DEP', 'MY'],
    columns='MONTH',
    values=['evi']
)

evi_pivot.columns = [ # rename columns with month number
    f"{col[0]}{col[1]}" if isinstance(col, tuple) and col[1] != "" 
    else col for col in evi_pivot.columns
]
evi_pivot = evi_pivot.reset_index()

Soil Organic Matter data (Particulate organic matter (POM) and Mineral-associated organic matter (MAOM))

In [179]:
som_pom = pd.read_json(f"{SOM_DATA_URL}/pom.json") #read json
som_pom['DEP'] = som_pom['nom'].map(departements) #map dep name to dep codevpd['DEP'] = vpd['dep'].map(departements) #map dep name to dep code
som_pom = som_pom.drop("nom", axis=1) #remove unwanted dep name -> final som pom data

In [180]:
maom_pom = pd.read_json(f"{SOM_DATA_URL}/maom.json") #read json
maom_pom['DEP'] = maom_pom['nom'].map(departements) #map dep name to dep codevpd['DEP'] = vpd['dep'].map(departements) #map dep name to dep code
maom_pom = maom_pom.drop("nom", axis=1) #remove unwanted dep name -> final som maom data

In [181]:
awc = pd.read_json(f"{AWC_DATA_URL}/AWC.json") #read json
awc['DEP'] = awc['nom'].map(departements) #map dep name to dep codevpd['DEP'] = vpd['dep'].map(departements) #map dep name to dep code
awc = awc.drop("nom", axis=1) #remove unwanted dep name -> final AWC data

Merging datasets

In [182]:
result = yields.merge(weather_pivot, on=['DEP', 'MY'], how='left')
result = result.merge(vpd_pivot, on=['DEP', 'MY'], how='left')
result = result.merge(evi_pivot, on=['DEP', 'MY'], how='left')

result = result.merge(som_pom, on="DEP", how="left")
result = result.merge(maom_pom, on="DEP", how="left")
result = result.merge(awc, on="DEP", how="left")

In [183]:
result = result.dropna(subset=['yield'])

In [184]:
result #show results

,year,yield,area,production,DEP,MY,RR1,RR2,RR3,RR4,...,evi6,evi7,evi8,evi9,evi10,evi11,evi12,pom,maom,awc
0,1980,4.39805,36001.0,158334.3,1,1979/1980,3.672457,3.484615,4.661456,1.224872,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.789692,26.925089,0.107088
1,1981,4.30319,36075.0,155237.5,1,1980/1981,5.279117,2.298026,4.151613,0.824649,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.789692,26.925089,0.107088
2,1982,5.00000,38050.0,190250.0,1,1981/1982,3.686452,0.559464,3.402339,0.685812,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.789692,26.925089,0.107088
3,1983,3.29500,37924.0,124959.6,1,1982/1983,3.015110,3.857753,3.601241,8.489750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.789692,26.925089,0.107088
4,1984,5.90000,40000.0,236000.0,1,1983/1984,5.722266,3.942845,3.163952,1.107667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.789692,26.925089,0.107088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4027,2020,0.35000,72.0,252.0,20,2019/2020,1.687465,0.514343,2.962535,3.276762,...,0.435666,0.431090,0.388402,0.355846,0.336295,0.313358,0.300604,15.657990,28.322897,0.109897
4028,2021,0.65000,99.0,643.5,20,2020/2021,5.630784,2.620412,1.022414,1.969150,...,0.405275,0.392680,0.360204,0.371781,0.375880,0.340750,0.330526,15.657990,28.322897,0.109897
4029,2022,0.65000,83.0,539.5,20,2021/2022,0.921650,0.828915,1.524876,3.056282,...,0.403204,0.379189,0.366802,0.343552,0.334657,0.288043,0.291791,15.657990,28.322897,0.109897
4030,2023,0.40000,63.0,252.0,20,2022/2023,4.384367,2.767651,1.613400,1.570661,...,0.417501,0.382688,0.369777,0.350959,0.324504,0.316170,0.319895,15.657990,28.322897,0.109897


In [185]:
result.to_csv("wheat_model_dataset_1980_2024.csv", index=False) #dataset to csv ! 